# Data and Model Exploration

This dataframe explores the dataset collected and tests different models to see which model may be best suited for predicting presence with the recorded scenario.

In [1]:
# set working directoy to root directory of the project
if not "working_directory_corrected" in vars():
    %cd ..
    %cd ..
    %cd ..
    working_directory_corrected = True
    
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import metrics

c:\Users\frank\Documents\Research\projects\evohome\environment\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


c:\Users\frank\Documents\Research\projects\evohome\examples\simple_scenario
c:\Users\frank\Documents\Research\projects\evohome\examples
c:\Users\frank\Documents\Research\projects\evohome


## Data Exploration

Below we load the dataset and observe it's general properties:

In [ ]:
data_frame = pd.read_pickle("examples/simple_scenario/data/recording.pickle")
nr_rooms = (len(data_frame.columns) - 3) // 2

print(f"The dataframe has {len(data_frame)} samples.")
print(f"The example scenario has {nr_rooms} rooms.")
print("below are five random samples")
data_frame

The dataframe has 259200 samples.
The example scenario has 11 rooms.
below are five random samples


,date,room_0,prediction_0,room_1,prediction_1,room_2,prediction_2,room_3,prediction_3,room_4,...,prediction_7,room_8,prediction_8,room_9,prediction_9,room_10,prediction_10,weekday,hour,minute
74065,2020-07-22 10:26:00,0,None,0,None,0,None,0,None,0,...,None,0,None,0,None,1,None,2,2,26
44271,2020-07-01 17:52:00,0,None,0,None,0,None,0,None,0,...,None,1,None,0,None,0,None,2,2,52
34355,2020-06-24 20:36:00,0,None,0,None,0,None,1,None,0,...,None,1,None,0,None,1,None,2,2,36
240469,2020-11-14 23:50:00,0,None,0,None,1,None,1,None,1,...,None,0,None,0,None,0,None,5,5,50
64134,2020-07-15 12:55:00,0,None,0,None,0,None,0,None,0,...,None,0,None,0,None,1,None,2,2,55


Below we calculate the utilization per room. This will tell us how often a presence is detected in the respective rooms in our data.

In [3]:
nr_samples = len(data_frame)
for room_index in range(nr_rooms):
    column = "room_" + str(room_index)
    nr_detections = (data_frame[column].values == 1).sum()
    utilization = int(100* round(nr_detections / nr_samples,2))
    print(f"Room room_{room_index} utilization: {utilization}%")

Room room_0 utilization: 0%
Room room_1 utilization: 0%
Room room_2 utilization: 47%
Room room_3 utilization: 49%
Room room_4 utilization: 45%
Room room_5 utilization: 0%
Room room_6 utilization: 0%
Room room_7 utilization: 4%
Room room_8 utilization: 21%
Room room_9 utilization: 17%
Room room_10 utilization: 41%


## Model Training

Here we test several machine learning models for their ability to learn this data.

Before we test, we split the data into training and test sets

In [4]:
training_features = ["weekday","hour", "minute"]
predicted_features = []
for index in range(nr_rooms):
    predicted_features.append("room_" + str(index))

X = data_frame[training_features]
y = data_frame[predicted_features]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(f"Number Training Samples: {len(X_train)}")
print(f"Number Test Samples: {len(X_test)}")



Number Training Samples: 207360
Number Test Samples: 51840


In [5]:
from pandas import DataFrame


model = tree.DecisionTreeClassifier()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)



In [6]:
print("Accuracy:",metrics.accuracy_score(y_pred=y_pred, y_true=y_test))
predictions_per_room = y_pred.transpose()
for index in range(room_index):
    accuracy = metrics.accuracy_score(y_pred = predictions_per_room[index], y_true = y_test["room_" + str(index)])
    print(f"- Room {index} Accuracy: {accuracy}")


Accuracy: 0.07903163580246914
- Room 0 Accuracy: 1.0
- Room 1 Accuracy: 1.0
- Room 2 Accuracy: 0.5694444444444444
- Room 3 Accuracy: 0.5279706790123457
- Room 4 Accuracy: 0.5732831790123457
- Room 5 Accuracy: 1.0
- Room 6 Accuracy: 1.0
- Room 7 Accuracy: 0.9646412037037037
- Room 8 Accuracy: 0.7892939814814814
- Room 9 Accuracy: 0.8306712962962963


In [7]:
model.predict([[6,22,59]])

c:\Users\frank\Documents\Research\projects\evohome\environment\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array([[0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0]])